In [23]:
import pandas as pd

# Load the dataset
data = pd.read_csv('zillow_data.csv')

# Display the first few rows of the dataset to understand its structure
data.head()


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


It contains columns for geographic identifiers (like RegionID, RegionName, City, State, Metro, CountyName, SizeRank) and numerous columns representing monthly house prices over several years.

In [24]:
# Function to convert the dataset from wide format to long format
def convert_to_long_format(df):
    # Melting the dataframe to convert it from wide to long format
    # Keeping the first 7 columns as id_vars, the rest are treated as value_vars
    df = data.melt(id_vars=df.columns[:7], var_name='Date', value_name='Price')
    
    # Converting 'Date' from string to datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Setting 'Date' as the index
    df.set_index('Date', inplace=True)
    
    return df

# Convert the dataset
data_long = convert_to_long_format(data)

# Display the first few rows of the long format dataset
data_long.head()


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,Price
Date,,,,,,,,
1996-04-01,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0
1996-04-01,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0
1996-04-01,91982,77494,Katy,TX,Houston,Harris,3,210400.0
1996-04-01,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0
1996-04-01,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0


In [25]:
most_recent_data = data_long['2008-01-01':]

most_recent_data

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,Price
Date,,,,,,,,
2008-01-01,84654,60657,Chicago,IL,Chicago,Cook,1,881700.0
2008-01-01,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,213900.0
2008-01-01,91982,77494,Katy,TX,Houston,Harris,3,254100.0
2008-01-01,84616,60614,Chicago,IL,Chicago,Cook,4,1163000.0
2008-01-01,93144,79936,El Paso,TX,El Paso,El Paso,5,131200.0
...,...,...,...,...,...,...,...,...
2018-04-01,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,209300.0
2018-04-01,59107,3293,Woodstock,NH,Claremont,Grafton,14720,225800.0
2018-04-01,75672,40404,Berea,KY,Richmond,Madison,14721,133400.0


In [26]:
# Checking for missing values and finding unique values
missing_values = most_recent_data.isnull().sum()
unique_values = most_recent_data.nunique()

missing_values

RegionID           0
RegionName         0
City               0
State              0
Metro         129332
CountyName         0
SizeRank           0
Price          26266
dtype: int64

***Within the dataset for the most recent 10 years, we have found the following:***

- There are 129,332 missing values in the 'Metro' column, which may indicate that some regions do not belong to a metropolitan area or that this information is not available.
- There are 26,266 missing values in the 'Price' column, suggesting that for some months in certain regions, the house prices are not recorded.
- No missing values are found in the other columns.

In [27]:
unique_values

RegionID      14723
RegionName    14723
City           7554
State            51
Metro           701
CountyName     1212
SizeRank      14723
Price         22003
dtype: int64

***Regarding the unique values for each column within this time frame:***


- 'RegionID': 14,723 unique region identifiers.
- 'RegionName': 14,723 unique region names.
- 'City': 7,554 unique cities.
- 'State': 51 unique states.
- 'Metro': 701 unique metropolitan areas.
- 'CountyName': 1,212 unique county names.
- 'SizeRank': 14,723 unique size ranks.
- 'Price': 22,003 unique price values.
- 'Year': 11 unique years, which is expected as we considered a 10-year range and some data may spill over into a eleventh year.


In [28]:
# Handling missing values in the 'Price' column
# We'll use linear interpolation as it's a common method for time-series data
most_recent_data['Price'] = most_recent_data['Price'].interpolate(method='linear')

# Handling missing values in the 'Metro' column by categorizing missing data as 'Non-Metro'
most_recent_data['Metro'] = most_recent_data['Metro'].fillna('Non-Metro')

# Checking if the missing values have been handled
missing_values_after = most_recent_data.isnull().sum()

missing_values_after

/tmp/ipykernel_1425/3289101980.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_recent_data['Price'] = most_recent_data['Price'].interpolate(method='linear')
/tmp/ipykernel_1425/3289101980.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_recent_data['Metro'] = most_recent_data['Metro'].fillna('Non-Metro')


RegionID      0
RegionName    0
City          0
State         0
Metro         0
CountyName    0
SizeRank      0
Price         0
dtype: int64

In [29]:
processed_data = most_recent_data 

In [33]:
def top_zipcodes_by_price_increment(data, price_col='Price', zipcode_col='RegionName', top_n=5):
    """
    Function to find the top N zipcodes with the highest price increment.

    :param data: DataFrame containing the real estate data.
    :param price_col: Name of the column containing price data.
    :param zipcode_col: Name of the column containing zipcode data.
    :param top_n: Number of top zipcodes to return based on price increment.
    :return: DataFrame containing the top N zipcodes and their price increments.
    """
    # Calculate price increment for each zipcode
    price_increment = data.groupby(zipcode_col).apply(lambda x: x[price_col].iloc[-1] - x[price_col].iloc[0])

    # Sort zipcodes by price increment and select top N
    top_zipcodes = price_increment.sort_values(ascending=False).head(top_n)

    return top_zipcodes


top_5_zipcodes = top_zipcodes_by_price_increment(processed_data)

top_5_zipcodes


RegionName
10021    7781500.0
10011    5791500.0
94027    3290300.0
10014    3181700.0
90210    2960500.0
dtype: float64